# **00 Import & Install Library**

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from textblob import Word

# **01 Load Dataset**

In [2]:
# job dataset
url_job = r"C:\Users\hp\COURSE\SISTECH\PP_MachineLearningOperations_TalithaRahmadewatiW\FINAL PROJECT\Dataset\jobstreet_data.csv"
df_job = pd.read_csv(url_job)

# course dataset
# url_course = r"C:\Users\hp\COURSE\SISTECH\PP_MachineLearningOperations_TalithaRahmadewatiW\FINAL PROJECT\Dataset\classcentral.csv"
# df_course = pd.read_csv(url_course)


# Load slang dictionary
url_slang = "https://raw.githubusercontent.com/talitharhmd/jobstreet-scraper/main/slang.csv"
df_slang = pd.read_csv(url_slang)
slang_dict = dict(zip(df_slang['slang'], df_slang['formal']))
additional_slang = {}  
slang_dict.update(additional_slang)

# Load stopword 
url_stopwords = "https://raw.githubusercontent.com/talitharhmd/jobstreet-scraper/main/stopword.csv"
stopword_manual = pd.read_csv(url_stopwords, header=None)
custom_stopwords = set(stopword_manual.iloc[:, 0].str.lower())
custom_stopwords.update([]) 

In [3]:
data = [
    {
        "Title": "CS50's Introduction to Computer Science",
        "Provider": "edX",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.59,
        "Price Type": "Free",
        "Reviews": "198 reviews",
        "Overview": "An entry-level course teaching core concepts of computer science, algorithms, data structures, and C programming.",
        "Skills": "C, algorithms, data structures, problem solving"
    },
    {
        "Title": "Introduction to Computer Science and Programming Using Python",
        "Provider": "edX",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.49,
        "Price Type": "Free",
        "Reviews": "131 reviews",
        "Overview": "Learn basic programming concepts such as loops, conditionals, and functions using Python.",
        "Skills": "Python, loops, conditionals, debugging"
    },
    {
        "Title": "Harvard CS50 – Full Computer Science University Course",
        "Provider": "freeCodeCamp",
        "Language": "English",
        "Certificate": "No Certificate",
        "Average Rating": 4.73,
        "Price Type": "Free",
        "Reviews": "834 reviews",
        "Overview": "A full university-level course that covers software engineering fundamentals, systems, and web development.",
        "Skills": "C, web development, systems programming, algorithms"
    },
    {
        "Title": "Introduction to Electrical Engineering and Computer Science I",
        "Provider": "MIT OpenCourseWare",
        "Language": "English",
        "Certificate": "No Certificate",
        "Average Rating": 4.68,
        "Price Type": "Free",
        "Reviews": "47 reviews",
        "Overview": "Covers basic principles of electrical engineering, computation, and signals.",
        "Skills": "circuits, digital logic, Python, signals"
    },
    {
        "Title": "CS50's Computer Science for Business Professionals",
        "Provider": "edX",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.76,
        "Price Type": "Free",
        "Reviews": "21 reviews",
        "Overview": "Designed for managers and business leaders to understand computational thinking and tech decisions.",
        "Skills": "business strategy, databases, web basics"
    },
    {
        "Title": "Computer Science 101",
        "Provider": "edX",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.11,
        "Price Type": "Free",
        "Reviews": "19 reviews",
        "Overview": "Introductory course to computing, binary numbers, hardware, and internet basics.",
        "Skills": "binary, computing, hardware, internet"
    },
    {
        "Title": "Internet History, Technology, and Security",
        "Provider": "Coursera",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.64,
        "Price Type": "Free",
        "Reviews": "42 reviews",
        "Overview": "Explores the history and evolution of the internet and key security concepts.",
        "Skills": "internet, TCP/IP, cryptography, security"
    },
    {
        "Title": "Computational Social Science Methods",
        "Provider": "Coursera",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.71,
        "Price Type": "Free",
        "Reviews": "138 reviews",
        "Overview": "Study how computational tools are used in social science to analyze data and behavior.",
        "Skills": "data analysis, network analysis, Python"
    },
    {
        "Title": "Functional Programming Principles in Scala",
        "Provider": "Coursera",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 4.60,
        "Price Type": "Free",
        "Reviews": "65 reviews",
        "Overview": "Learn the fundamentals of functional programming using the Scala programming language.",
        "Skills": "Scala, functional programming, recursion"
    },
    {
        "Title": "CS50's Computer Science for Lawyers",
        "Provider": "edX",
        "Language": "English",
        "Certificate": "Certificate Available",
        "Average Rating": 5.00,
        "Price Type": "Free",
        "Reviews": "8 reviews",
        "Overview": "A course that teaches legal professionals the foundations of computer science relevant to law and technology.",
        "Skills": "cybersecurity, privacy, law & tech, logic"
    },
]

df_courses = pd.DataFrame(data)


# **02 Overview** 

In [4]:
## Data set information for Jobstreet
print("**"*30)
print(" "*15, "Information Jobsteet dataset")
print("**"*30)
print(df_job.info())

************************************************************
                Information Jobsteet dataset
************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          480 non-null    object
 1   country      480 non-null    object
 2   title        480 non-null    object
 3   company      480 non-null    object
 4   location     480 non-null    object
 5   category     480 non-null    object
 6   work_type    480 non-null    object
 7   description  480 non-null    object
dtypes: object(8)
memory usage: 30.1+ KB
None


In [5]:
## Data set information for Classentral
print("**"*30)
print(" "*15, "Information Classentral dataset")
print("**"*30)
print(df_courses.info())

************************************************************
                Information Classentral dataset
************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           10 non-null     object 
 1   Provider        10 non-null     object 
 2   Language        10 non-null     object 
 3   Certificate     10 non-null     object 
 4   Average Rating  10 non-null     float64
 5   Price Type      10 non-null     object 
 6   Reviews         10 non-null     object 
 7   Overview        10 non-null     object 
 8   Skills          10 non-null     object 
dtypes: float64(1), object(8)
memory usage: 848.0+ bytes
None


In [6]:
df_courses.head(10)

,Title,Provider,Language,Certificate,Average Rating,Price Type,Reviews,Overview,Skills
0,CS50's Introduction to Computer Science,edX,English,Certificate Available,4.59,Free,198 reviews,An entry-level course teaching core concepts o...,"C, algorithms, data structures, problem solving"
1,Introduction to Computer Science and Programmi...,edX,English,Certificate Available,4.49,Free,131 reviews,Learn basic programming concepts such as loops...,"Python, loops, conditionals, debugging"
2,Harvard CS50 – Full Computer Science Universit...,freeCodeCamp,English,No Certificate,4.73,Free,834 reviews,A full university-level course that covers sof...,"C, web development, systems programming, algor..."
3,Introduction to Electrical Engineering and Com...,MIT OpenCourseWare,English,No Certificate,4.68,Free,47 reviews,Covers basic principles of electrical engineer...,"circuits, digital logic, Python, signals"
4,CS50's Computer Science for Business Professio...,edX,English,Certificate Available,4.76,Free,21 reviews,Designed for managers and business leaders to ...,"business strategy, databases, web basics"
5,Computer Science 101,edX,English,Certificate Available,4.11,Free,19 reviews,"Introductory course to computing, binary numbe...","binary, computing, hardware, internet"
6,"Internet History, Technology, and Security",Coursera,English,Certificate Available,4.64,Free,42 reviews,Explores the history and evolution of the inte...,"internet, TCP/IP, cryptography, security"
7,Computational Social Science Methods,Coursera,English,Certificate Available,4.71,Free,138 reviews,Study how computational tools are used in soci...,"data analysis, network analysis, Python"
8,Functional Programming Principles in Scala,Coursera,English,Certificate Available,4.60,Free,65 reviews,Learn the fundamentals of functional programmi...,"Scala, functional programming, recursion"
9,CS50's Computer Science for Lawyers,edX,English,Certificate Available,5.00,Free,8 reviews,A course that teaches legal professionals the ...,"cybersecurity, privacy, law & tech, logic"


# **03 Text Preprocessing**

In [19]:
# 1. lowercase
def lowercase_columns(df):
    df.columns = df.columns.str.lower()  # Ubah nama kolom
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.lower()
    return df

# 2. clean text
def clean_text(text):
    if not isinstance(text, str): return ""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[\n\r\t]+", " ", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return np.nan if text == "" else text

# 3. translate to english
def translate_text(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return text 

# 4. replace slang
def replace_slang(text):
    if not isinstance(text, str): return ""
    words = text.split()
    return " ".join([slang_dict.get(w, w) for w in words])

# 5. tokenize
def tokenizing_text(text):
    return word_tokenize(text)

# 6. 
def replace_slang(text):
    if not isinstance(text, str): return ""
    return " ".join([slang_dict.get(word, word) for word in text.split()])

stopwords_nltk = set(stopwords.words('english'))

def preprocess_pipeline(text):
    if not isinstance(text, str): return ""
    text = clean_text(text)
    text = replace_slang(text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in custom_stopwords and w not in stopwords_nltk]
    lemmatized = [Word(w).lemmatize("v") if Word(w).lemmatize("v") != w else Word(w).lemmatize("n") for w in tokens]
    return " ".join(lemmatized)

df_job = lowercase_columns(df_job)
df_courses = lowercase_columns(df_courses)

# 6. generate gabungan kolom & apply pipeline
for df, cols, name in [
    (df_job, ['title', 'category', 'description'], 'job'),
    (df_courses, ['title', 'overview', 'skills'], 'course')
]:
    df['text'] = df[cols].astype(str).apply(lambda x: ' '.join(x), axis=1)
    df['text_clean'] = df['text'].apply(preprocess_pipeline)

# Tokenisasi text_clean
df_job["tokens"] = df_job["text_clean"].astype(str).apply(word_tokenize)
df_courses["tokens"] = df_courses["text_clean"].astype(str).apply(word_tokenize)

## Pipeline

In [20]:
# === JOB DATA ===
df_job['text'] = df_job['title'].astype(str) + " " + df_job['category'].astype(str) + " " + df_job['description'].astype(str)
df_job['text_clean'] = df_job['text'].apply(preprocess_pipeline)

# === COURSE DATA ===
df_courses['text'] = df_courses['title'].astype(str) + " " + df_courses['overview'].astype(str) + " " + df_courses['skills'].astype(str)
df_courses['text_clean'] = df_courses['text'].apply(preprocess_pipeline)


In [21]:
df_job.head()

,url,country,title,company,location,category,work_type,description,text,text_clean,tokens
0,https://my.jobstreet.com/job/84998805?type=sta...,malaysia,"data scientist, financial conglomerates superv...",bank negara malaysia,kuala lumpur,analysis & reporting (banking & financial serv...,full time,role purpose:\nexecute the suptech and data an...,"data scientist, financial conglomerates superv...",data scientist financial conglomerate supervis...,"[data, scientist, financial, conglomerate, sup..."
1,https://my.jobstreet.com/job/85371197?type=sta...,malaysia,data scientist,western digital tech and regional center (m) s...,kuala lumpur,"mathematics, statistics & information sciences...",full time,job description\nresponsibilities\nwork with d...,"data scientist mathematics, statistics & infor...",data scientist mathematics statistic science s...,"[data, scientist, mathematics, statistic, scie..."
2,https://my.jobstreet.com/job/85548582?type=sta...,malaysia,data annotator,concentrix,"cyberjaya, selangor",database development & administration (informa...,full time,job description\nkey responsibilities:\n accur...,data annotator database development & administ...,data annotator database development administra...,"[data, annotator, database, development, admin..."
3,https://my.jobstreet.com/job/85154961?type=sta...,malaysia,data scientist (artificial intelligence),dcap commercial sdn bhd,kuala lumpur,"mathematics, statistics & information sciences...",full time,"why join us? \nat dcap, we’re dynamic, fast-gr...",data scientist (artificial intelligence) mathe...,data scientist artificial intelligence mathema...,"[data, scientist, artificial, intelligence, ma..."
4,https://my.jobstreet.com/job/84832618?type=sta...,malaysia,data scientist,msa focus international ltd,"kelana jaya, selangor","mathematics, statistics & information sciences...",full time,key responsibilities\ndata exploration & model...,"data scientist mathematics, statistics & infor...",data scientist mathematics statistic science s...,"[data, scientist, mathematics, statistic, scie..."


## Saving Cleaned Dataset 

In [22]:
df_job.to_csv("cleaned_jobstreet.csv", index=False)
df_courses.to_csv("cleaned_classentral.csv", index=False)